In [1]:
# Python : 3.10.16
# Created: Mar. 21. 2025
# Updated: Mar. 21. 2025
# Author: D.W. SHIN
# Description: 미스트랄 7B 모델을 활용하여 로컬 PC에서 금융 데이터 파인튜닝하기
# Huggingface와 WandB를 이용합니다.
#
# 참고문서 : 도메인 특화 LLM: Mistral 7B를 활용한 금융 업무분야 파인튜닝 및 활용 방법 (정천수)
# 데이터셋 출처 : https://huggingface.co/datasets/csujeong/Non_life_insurance

In [2]:
# 필요 라이브러리 설치
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 && \
# pip install transformers datasets accelerate peft bitsandbytes trl wandb huggingface_hub

In [3]:
# 📌 1. 라이브러리 로드
import pandas as pd
import torch
import wandb
from datasets import Dataset
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

In [4]:
# 📌 2. 데이터 로드 (로컬 CSV 파일 사용)
DATA_PATH = "./data/Non-life_insurance_Dataset.csv"
df = pd.read_csv(DATA_PATH)
df = df.dropna()  # NaN 값 제거
dataset = Dataset.from_pandas(df)

In [5]:
# !huggingface-cli login

In [6]:
# 📌 3. 모델 로드
model_name = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 🔥 로그인한 Hugging Face 토큰 사용
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    token=True,
)

tokenizer = AutoTokenizer.from_pretrained(model_name, token=True)
tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# 📌 4. LoRA 설정 및 적용
lora_alpha = 32
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj"],
)

peft_model = get_peft_model(model, peft_config)

In [8]:
# !wandb login

In [9]:
# 📌 5. WandB 설정 (보안 강화 및 `entity` 추가)
wandb.login()  # 🔥 API Key를 환경변수에 설정하지 않고 로그인 수행

wandb.init(
    project="Mistral-7B-Finance",
    name="qlora_finetuning",
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: dongweon-shin (dongweonshin) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [10]:
# 📌 6. 훈련 설정
training_args = TrainingArguments(
    output_dir="./models/Mistral-7B-Finetuned",  # 로컬 모델 저장 경로
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    save_strategy="steps",
    save_steps=10,
    logging_steps=10,
    learning_rate=2e-4,
    max_grad_norm=0.3,
    max_steps=60,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    push_to_hub=False,
)

In [11]:
# `formatting_func` 수정 (데이터를 올바른 string 형식으로 변환)
def formatting_func(example):
    return {"text": str(example["QA_text"]).strip()}  # 문자열로 변환 후 공백 제거


# 데이터셋을 변환하여 올바르게 정리
dataset = dataset.map(formatting_func)

Map:   0%|          | 0/545 [00:00<?, ? examples/s]

In [12]:
# 📌 7. Trainer 생성
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=dataset,
    peft_config=peft_config,
    args=training_args,
)

Converting train dataset to ChatML:   0%|          | 0/545 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/545 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/545 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/545 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [13]:
# 📌 8. 학습 실행
peft_model.config.use_cache = False  # 캐시 비활성화
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
10,1.820600
20,1.454200
30,1.497700
40,1.325100
50,1.375000
60,1.348200


TrainOutput(global_step=60, training_loss=1.4701295375823975, metrics={'train_runtime': 36.5596, 'train_samples_per_second': 6.565, 'train_steps_per_second': 1.641, 'total_flos': 1941095228866560.0, 'train_loss': 1.4701295375823975})

In [14]:
# 📌 9. 모델 저장
trainer.save_model("./models/Mistral-7B-Finetuned")
tokenizer.save_pretrained("./models/Mistral-7B-Finetuned")

('./models/Mistral-7B-Finetuned/tokenizer_config.json',
 './models/Mistral-7B-Finetuned/special_tokens_map.json',
 './models/Mistral-7B-Finetuned/tokenizer.json')

In [15]:
# 📌 10. 파인튜닝된 모델 불러오기
PEFT_MODEL = "./models/Mistral-7B-Finetuned"

config = PeftConfig.from_pretrained(PEFT_MODEL)
peft_base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)

peft_model = PeftModel.from_pretrained(peft_base_model, PEFT_MODEL)
peft_tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
peft_tokenizer.pad_token = peft_tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
# 📌 11. 테스트 (질문-응답 생성 함수)
def generate_answer(prompt):
    inputs = peft_tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        output = peft_model.generate(**inputs, max_length=512)
    return peft_tokenizer.decode(output[0], skip_special_tokens=True)

In [17]:
# 📌 12. 테스트 실행
print(generate_answer("골프보험 알려줘"))
print(generate_answer("선물이 뭐야?"))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


골프보험 알려줘? 골프보험은 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골프를 하는 중에 발생하는 사고를 보상하는 보험입니다. 골프장에서 골
선물이 뭐야? 선물은 미래 가입계약으로 미래 가입계약은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선물은 미래 가입계약이라는 말 그대로 미래에 가입하는 계약을 말합니다. 선


# 📊 학습 결과 분석 (wandb)
모델의 학습 과정과 GPU 성능 모니터링 데이터를 wandb 대시보드에서 확인할 수 있습니다.

## 🔹 학습 과정 (Training Metrics)
아래의 그래프는 모델 훈련 중 손실(loss), 정확도(accuracy), 학습률(learning rate) 등의 변화를 보여줍니다.

![Training Metrics](images/train_metrics.png)

- **Loss 감소**: 학습이 정상적으로 진행되면서 모델이 점진적으로 개선됨을 보여줌.
- **Mean Token Accuracy 증가**: 모델이 점점 더 정확한 출력을 생성하고 있음.
- **Learning Rate 감소**: 안정적인 학습률 스케줄을 따르면서 수렴 중.

## 🔹 GPU 성능 모니터링 (System Monitoring)
아래의 그래프는 학습 중 GPU 리소스 사용량을 나타냅니다.

![System Monitoring](images/system_monitoring.png)

- **GPU 사용량 변동**: 학습 과정에서 GPU 자원을 효율적으로 활용함.
- **GPU 클럭 속도 변화**: 모델 학습 중 적절한 연산 성능을 유지.
- **전력 제한 준수**: GPU의 전력 제한이 초과되지 않으며, 시스템 안정성이 유지됨.

📌 **결론**: 모델 학습이 정상적으로 이루어졌으며, GPU 리소스가 효과적으로 활용됨을 확인할 수 있습니다. 추가적인 튜닝을 원할 경우, `learning_rate`, `batch_size`, `gradient_accumulation_steps` 등을 조정하여 실험을 진행할 수 있습니다.
